<a href="https://colab.research.google.com/github/CODERZZZ123/StockPred_NewsHeadlines/blob/master/pred_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
import numpy as np
import pandas as pd
import nltk.corpus
import re
nltk.download('stopwords')
nltk.download('wordnet')


from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt
!pip install sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import euclidean_distances
!pip install textattack

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [45]:
from google.colab import drive
drive.mount('/content/drive')
# import requests
# import time

# while True:
#     try:
#         requests.get('https://www.google.com')
#         print("Kept alive.")
#     except:
#         print("Failed to keep alive.")
#     time.sleep(600)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
from sklearn import  set_config
set_config(display = 'diagram')

In [47]:
df_NewsDjia = pd.read_csv('News_DJIA.csv', delimiter=',')
df_RedditNews = pd.read_csv('News_reddit.csv', delimiter=',')
df_DjiaStock = pd.read_csv('DJIA_t.csv',delimiter=',')

In [48]:
for i in df_NewsDjia.columns :
    df_NewsDjia[i] = df_NewsDjia[i].astype(str)

In [49]:
df_NewsDjia

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [50]:
num_columns = df_NewsDjia.columns.shape[0]
col_names = df_NewsDjia.columns.values.tolist()
col_names = col_names[1:]
df_NewsDjia.loc[:, 'merged'] = " "
for each_col_ind in range(num_columns-1):
    df_NewsDjia.loc[:, 'merged'] =  df_NewsDjia.loc[:, 'merged'] + " " + df_NewsDjia[col_names[each_col_ind]] + " "

In [51]:
df_finalised = df_NewsDjia[['merged']]

In [52]:
date_numric_stopwords = "THOUSAND MILLION BILLION TRILLION HUNDRED ANNUAL ANNUALLY ANNUM YEAR YEARLY QUARTER QUARTERLY QTR MONTH MONTHLY WEEK WEEKLY DAY DAILY JANUARY FEBRUARY MARCH APRIL MAY JUNE JULY AUGUST SEPTEMBER OCTOBER NOVEMBER DECEMBER JAN FEB MAR APR MAY JUN JUL AUG SEP SEPT OCT NOV DEC MONDAY TUESDAY WEDNESDAY THURSDAY FRIDAY SATURDAY SUNDAY TWO THREE FOUR FIVE SIX SEVEN EIGHT NINE TEN ELEVEN TWELVE THIRTEEN FOURTEEN FIFTEEN SIXTEEN SEVENTEEN EIGHTEEN NINETEEN TWENTY THIRTY FORTY FIFTY SIXTY SEVENTY EIGHTY NINETY FIRST SECOND THIRD FOURTH FIFTH SIXTH SEVENTH EIGHTH NINTH TENTH II III IV V VI VII VIII IX X XI XII XIII XIV XV XVI XVII XVIII XIX XX"
date_numric_stopwords = date_numric_stopwords.lower()
date_numric_stopwords = date_numric_stopwords.split(" ")

In [53]:

def remove_unecssary_word(sent):
    sent = sent.replace("b'",'')
    sent = sent.replace('b"','')
    sent = sent.lower()
    sent = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", sent)
    stop = stopwords.words('english')
    stop.extend(date_numric_stopwords)
    sent = " ".join([word for word in sent.split() if word not in (stop)])
    sent = " ".join([WordNetLemmatizer().lemmatize(word) for word in sent.split() ])
    return sent

In [54]:
df_finalised['merged'] = df_finalised['merged'].apply(remove_unecssary_word)

<ipython-input-54-6847205c14ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_finalised['merged'] = df_finalised['merged'].apply(remove_unecssary_word)


In [55]:
df_finalised.iloc[0][0]

'0 georgia down russian warplane country move brink war breaking musharraf impeached russia today column troop roll south ossetia footage fighting youtube russian tank moving towards capital south ossetia reportedly completely destroyed georgian artillery fire afghan child raped impunity un official say sick old raped nothing 150 russian tank entered south ossetia whilst georgia shoot russian jet breaking georgia invades south ossetia russia warned would intervene so side enemy combatent trial nothing sham salim haman sentenced 5 12 year kept longer anyway feel like georgian troop retreat osettain capital presumably leaving several people killed video u prep georgia war russia rice give green light israel attack iran say u veto israeli military ops announcingclass action lawsuit behalf american public fbi sorussia georgia war nyts top story opening ceremony olympics fucking disgrace yet proof decline journalism china tell bush stay country affair world war start today georgia invades s

In [56]:
# from textattack.augmentation import EmbeddingAugmenter
# embed_aug = EmbeddingAugmenter()
# embed_aug.augment(df_finalised.iloc[0][0])

In [57]:
# from textattack.augmentation import WordNetAugmenter
# wordnet_aug = WordNetAugmenter()
# wordnet_aug.augment(df_finalised.iloc[0][0])

In [58]:
# from textattack.augmentation import EasyDataAugmenter
# eda_aug = EasyDataAugmenter()
# eda_aug.augment(df_finalised.iloc[0][0])

In [59]:
df_finalised = pd.concat([df_finalised,df_NewsDjia['Label']],axis = 1)

In [60]:
Positive_word = ""
with open('Positive_words.txt', 'r') as file:
    Positive_word = file.read().replace('\n', '')
Positive_word = " ".join([WordNetLemmatizer().lemmatize(word) for word in Positive_word.split(',') ])

In [61]:
Negative_word = ""
with open('Negative_words.txt','r') as file:
    Negative_word = file.read().replace('\n','')
Negative_word = " ".join([WordNetLemmatizer().lemmatize(word) for word in Negative_word.split(',') ])

In [62]:
Positive_word = Positive_word.split(',')
Negative_word = Negative_word.split(',')
Negative_word = Negative_word[0].split()
Positive_word = Positive_word[0].split()

In [63]:
set_post = set(Positive_word)
set_neg = set(Negative_word)
def Count(obj):
    count_post = 1
    count_neg = 1
    obj = obj.split()
    for i in obj:
        if i in set_post:
            count_post = count_post + 1
        elif i in set_neg:
            count_neg = count_neg + 1
    return count_post/count_neg


In [64]:
df_finalised['ratio_pos_neg'] = df_finalised['merged'].apply(Count)
X_data = df_finalised['merged']
# ratio_column = df_finalised['ratio_pos_neg']
# ratio_column = np.array(ratio_column).reshape((-1,1))

In [65]:
Y_data = df_finalised['Label']
Y_data = Y_data.astype(int)

In [66]:
from textattack.augmentation import EasyDataAugmenter
eda_aug = EasyDataAugmenter()
data_X = []
label = []
# open('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/data.txt', 'w').close()
# open('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/label.txt','w').close()
with open('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/data.txt', 'a') as f1 , open('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/label.txt','a') as f2:
  for i in range(1960,X_data.shape[0]):
    data_X.append(X_data[i])
    label.append(Y_data[i])
    f1.write(X_data[i])
    f1.write(",")
    y = str(Y_data[i])
    f2.write(y)
    f2.write(",")
    if i%10 == 0:
      data = eda_aug.augment(X_data[i])
      for d in data:
        data_X.append(d)
        label.append(Y_data[i])
        f1.write(d)
        f1.write(",")
        f2.write(y)
        f2.write(",")

    print(i)

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


KeyboardInterrupt: ignored

In [67]:
Y_data = np.array(Y_data)
Y_data

array([0, 1, 0, ..., 1, 1, 1])

In [68]:
X_data[0]

'0 georgia down russian warplane country move brink war breaking musharraf impeached russia today column troop roll south ossetia footage fighting youtube russian tank moving towards capital south ossetia reportedly completely destroyed georgian artillery fire afghan child raped impunity un official say sick old raped nothing 150 russian tank entered south ossetia whilst georgia shoot russian jet breaking georgia invades south ossetia russia warned would intervene so side enemy combatent trial nothing sham salim haman sentenced 5 12 year kept longer anyway feel like georgian troop retreat osettain capital presumably leaving several people killed video u prep georgia war russia rice give green light israel attack iran say u veto israeli military ops announcingclass action lawsuit behalf american public fbi sorussia georgia war nyts top story opening ceremony olympics fucking disgrace yet proof decline journalism china tell bush stay country affair world war start today georgia invades s

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [69]:
data_X

['0 keep happening rape survivor say protest american military presence japan continued country defense minister lodged formal complaint latest case alleged murder rape young japanese woman u military base employee obama apologize hiroshima attack tell japanese tv austria reject farright president climate denial argument fail blind test pepsi challenge test economist statistician find mainstream climate argument accurate contrarian argument wrong misleading operation retake fallujah begun iraqi prime minister say india firstever indigenous space shuttle launched successfully chinese russian warplane increasingly aggressive intercepting u military aircraft patrolling near america west coast prompting air force top combat officer label provocation one top worry isi threatens attack india isi mocked muslim living harmony hindu urged travel isisheld territoriesindia hindumajority country home 160 muslim handful joined middle eastern group turkey prime minister formally submitted resignatio

In [104]:
data_augmented = np.genfromtxt('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/data.txt',dtype = str , delimiter=",")

In [105]:
data_augmented.shape

(2864,)

In [106]:
count = 0
count2 = 0
for i in data_augmented:
    if len(i) != 0:
        if i[0] =='0' or i[0] == '1':
            if i[1] == " ":

              count = count + 1
    count2 = count2 + 1
print(count)
print(count2)

2788
2864


In [107]:
index = 0
for i in data_augmented:
  if len(i) != 0 :
    if i[0] != '1' and i[0] != '0' :
        print(index)
        print(data_augmented[index-1],'\n')
        print(i,'\n')
        data_augmented[index] = data_augmented[index-1][0] + " " + i
    index += 1

3
0 georgia down russian warplane country move brink war breaking musharraf impeached russia today column troop disgrace south ossetia footage fighting youtube russian tank would towards capital south ossetia reportedly completely destroyed georgian artillery fire afghan child raped impunity old official say sick un raped nothing 150 russian tank entered south ossetia whilst georgia war russian jet breaking georgia invades south ossetia russia bush moving intervene haman side enemy combatent trial nothing sham salim so sentenced 5 12 year kept longer anyway feel like georgian troop retreat osettain capital presumably leaving several people killed video u prep georgia war russia approved give illness light israel attack iran say u veto israeli case ops announcingclass action georgia behalf american public fbi prevent georgia shoot nyts top story opening ceremony olympics fucking roll yet proof decline journalism china indian warned stay country affair world war start today georgia invad

In [108]:
data_augmented = np.delete(data_augmented,2863,axis = 0)

In [109]:
data_augmented.shape

(2863,)

In [110]:
label_updated = data_augmented.astype('<U1')

In [111]:
label_updated

array(['0', '0', '0', ..., '1', '1', '0'], dtype='<U1')

In [142]:
label_updated = label_updated.astype(np.float)

<ipython-input-142-39bc489429a0>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  label_updated = label_updated.astype(np.float)


In [112]:
data_augmented = list(data_augmented)

In [113]:
data_augmented = [i[2:] for i in data_augmented]

In [114]:
data_augmented

['georgia down russian warplane country move brink war breaking musharraf impeached russia today column troop roll south ossetia footage fighting youtube russian tank moving towards capital south ossetia reportedly completely destroyed georgian artillery fire afghan child raped impunity un official say sick old raped nothing 150 russian tank entered south ossetia whilst georgia shoot russian jet breaking georgia invades south ossetia russia warned would intervene so side enemy combatent trial nothing sham salim haman sentenced 5 12 year kept longer anyway feel like georgian troop retreat osettain capital presumably leaving several people killed video u prep georgia war russia rice give green light israel attack iran say u veto israeli military ops announcingclass action lawsuit behalf american public fbi sorussia georgia war nyts top story opening ceremony olympics fucking disgrace yet proof decline journalism china tell bush stay country affair world war start today georgia invades so

In [70]:
X_data.shape

(1989,)

In [143]:
x_train , x_test , y_train, y_test = train_test_split(data_augmented,label_updated , test_size = 0.2 , random_state = 42)

In [144]:
tf_idf = TfidfVectorizer(min_df=10)
x_train_tf = tf_idf.fit_transform(x_train)
x_test_tf = tf_idf.transform(x_test)
x_train = x_train_tf.toarray()
x_test = x_test_tf.toarray()

In [145]:
mean_not_zero = x_train[x_train != 0].mean()
x_train[x_train!=0] = x_train[x_train!=0] -  mean_not_zero
x_test[x_test != 0 ] = x_test[x_test != 0] - mean_not_zero

In [146]:
# ct_vect = CountVectorizer()
# x_train = ct_vect.fit_transform(x_train).toarray()
# x_test = ct_vect.transform(x_test).toarray()

In [147]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [148]:
pca = PCA()
pca.fit(x_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)*100
d = [n for n in range(len(cumsum))]
plt.figure(figsize=(10, 10))
plt.plot(d,cumsum, color = 'red',label='cumulative explained variance')
plt.title('Cumulative Explained Variance as a Function of the Number of Components')
plt.ylabel('Cumulative Explained variance')
plt.xlabel('Principal components')
plt.axhline(y = 90, color='k', linestyle='--', label = '95% Explained Variance')
plt.legend(loc='best')

KeyboardInterrupt: ignored

In [149]:
# VARIANCE EXPLAINED 90%
pca = PCA(0.80)
pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)
pca.explained_variance_ratio_

array([0.00448458, 0.00378159, 0.00355305, 0.00339778, 0.00323897,
       0.00316912, 0.00307115, 0.00296073, 0.00293882, 0.00284112,
       0.00278231, 0.00274931, 0.00272087, 0.00267625, 0.0026662 ,
       0.00260887, 0.00259299, 0.00256171, 0.00249599, 0.00242392,
       0.0023825 , 0.00236446, 0.00233972, 0.00232387, 0.00230985,
       0.00230381, 0.00228477, 0.00225109, 0.00223547, 0.00223028,
       0.00222243, 0.00219814, 0.00218529, 0.00217053, 0.0021539 ,
       0.00214071, 0.00213628, 0.00212574, 0.00210202, 0.00207965,
       0.00204776, 0.00204142, 0.00202909, 0.00202128, 0.00201532,
       0.00200311, 0.00199806, 0.00198518, 0.00197252, 0.00195837,
       0.00194997, 0.00194111, 0.00193539, 0.00192425, 0.00192027,
       0.00191063, 0.00190401, 0.00189624, 0.00188422, 0.00187731,
       0.00186123, 0.00185152, 0.00184703, 0.00183834, 0.00183342,
       0.00182203, 0.00182034, 0.00181539, 0.00180732, 0.0017973 ,
       0.00179031, 0.00178496, 0.00177946, 0.00176764, 0.00176

In [ ]:
# K = [int(i) for i in range(x_train.shape[0]//3)]
# param_grid = {
#     'n_neighbors' : K
# }
# knn = KNeighborsClassifier()
# grid_search = GridSearchCV(estimator=knn,scoring='accuracy',param_grid=param_grid,cv = 10,n_jobs = -1,verbose = 2)
# grid_search.fit(x_train,y_train)
# grid_search.best_estimator_
# knn = KNeighborsClassifier(n_neighbors=1)
# knn.fit(x_train,y_train)
# y_pred = knn.predict(x_test)
# print(accuracy_score(y_pred,y_test))
# print(classification_report(y_pred,y_test))

In [ ]:
# C = [0.01 , 0.05  , 0.08, 0.1, 0.3 , 0.6 , 0.9 , 1.2 , 1.7 , 2.5 , 2.9 , 3.7, 4.6 , 5.1 , 5.9 , 6.9 , 7.5 , 8.3 , 9.3 , 10]
# kernel = ['linear', 'poly', 'rbf', 'sigmoid']
# gamma = ['scale','auto']
# degree = [int(x) for x in range(1,5)]
# param_grid = {
#     'kernel' : kernel, vg
#     'gamma' : gamma,
#     'degree' : degree,
#     'C' : C
# }
# svc = SVC()
# grid_search = GridSearchCV(estimator=svc,scoring='accuracy',param_grid=param_grid,cv = 5,n_jobs = -1,verbose = 2)
# grid_search.fit(x_train,y_train)
# best_gird_search = grid_search.best_estimator_
# from sklearn.metrics import accuracy_score
# y_pred = best_gird_search.predict(x_test)
# print(confusion_matrix(y_test,y_pred))
# print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
# print(accuracy_score(y_train,best_gird_search.predict(x_train)))
# Score = cross_val_score(best_gird_search,X_data,Y_data,cv = 5)
# Score

In [ ]:
# from sklearn.metrics import accuracy_score
# model = SVC(C = 25, degree= 4, gamma =  'auto', kernel = 'linear')
# model.fit(x_train,y_train)
# y_pred = model.predict(x_test)
# print(confusion_matrix(y_test,y_pred))
# print("Accuracy for the trained data is {}".format(accuracy_score(y_train,model.predict(x_train))))
# print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
# print("classification report {}".format(classification_report(y_test,y_pred)))
# # Score = cross_val_score(model,X_data,Y_data,cv = 5)
# Score

In [133]:
!pip install tensorflow

In [150]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# model = Sequential()
# model.add(Dense(12,input_shape = (x_train.shape[1],),activation = 'relu'))
# model.add(Dense(8,activation = 'relu'))
# model.add(Dense(1,activation = 'sigmoid'))
# model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# model.fit(x_train,y_train,epochs=200,batch_size=1000)
# predictions = (model.predict(x_test) > 0.5).astype(int)
# abs(predictions.T - y_test).sum()/y_test.shape[0]


Epoch 1/200
3/3 [==============================] - 1s 16ms/step - loss: 0.8026 - accuracy: 0.5201
Epoch 2/200
3/3 [==============================] - 0s 10ms/step - loss: 0.6968 - accuracy: 0.5948
Epoch 3/200
3/3 [==============================] - 0s 9ms/step - loss: 0.6241 - accuracy: 0.6716
Epoch 4/200
3/3 [==============================] - 0s 15ms/step - loss: 0.5671 - accuracy: 0.7223
Epoch 5/200
3/3 [==============================] - 0s 9ms/step - loss: 0.5219 - accuracy: 0.7620
Epoch 6/200
3/3 [==============================] - 0s 9ms/step - loss: 0.4834 - accuracy: 0.7961
Epoch 7/200
3/3 [==============================] - 0s 11ms/step - loss: 0.4503 - accuracy: 0.8223
Epoch 8/200
3/3 [==============================] - 0s 12ms/step - loss: 0.4222 - accuracy: 0.8511
Epoch 9/200
3/3 [==============================] - 0s 10ms/step - loss: 0.3971 - accuracy: 0.8659
Epoch 10/200
3/3 [==============================] - 0s 9ms/step - loss: 0.3742 - accuracy: 0.8786
Epoch 11/200
3/3 [=====

0.2617801047120419

In [151]:
randomclassifier=RandomForestClassifier(n_estimators=1000,criterion='entropy')
randomclassifier.fit(x_train,y_train)
predictions = randomclassifier.predict(x_test)
matrix=confusion_matrix(y_test,predictions)
print(matrix)
score=accuracy_score(y_test,predictions)
print(score)
report=classification_report(y_test,predictions)
print(report)

[[160 148]
 [ 12 253]]
0.7207678883071553
              precision    recall  f1-score   support

         0.0       0.93      0.52      0.67       308
         1.0       0.63      0.95      0.76       265

    accuracy                           0.72       573
   macro avg       0.78      0.74      0.71       573
weighted avg       0.79      0.72      0.71       573

